<div style="background:#E9FFF6; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFN619 - Data Analytics for Strategic Decision Makers (2023_sem1)</div>

# IFN619 :: C1-UnstructuredAnalytics

For this session, the focus will be on analysis of unstructured text. However, the thinking required is similar to approaches to analysing images, video, sound and other unstructured data. Primarily, the analysis is based on the notion that there are useful patterns in the unstructured data which can be obtained mathematically. By converting the data to a mathematical structure, various algorithms can be applied to the structure with the aim of identifying patterns. 

In the case of the `topic modelling` approaches below, many of the techniques are *probabilistic* - that is they mathematically identify the *likelihood* that a feature might be important. Thus, they are never 100% accurate, and their use needs to be mediated by a more pragmatic *useful or not* approach, rather than *right or wrong*.

In [ ]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pandas as pd
import json
import random

### Accessing the data via The Guardian API

See the `Accessing_the_Guardian_API.ipynb` notebook file for details on getting the data. **Note:** This approach may be used for additional data for Assignment 2.

### Read in pre-saved data

To save time, we're loading in pre-saved data that was fetched using the Guardian API.

In [ ]:
# Load the data - articles from The Guardian about the war in Ukraine
file_path = "data/"
file_name = ???

with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
    articles = json.load(fp)

print(f"Loaded {len(articles)} articles from {file_name}")

Each dictionary entry includes the *title [date]* as `key` and the *body text* from the article as `value`.

In [ ]:
article1 = list(articles.items())[0]
print("Key:",article1[0])
print("Value:",article1[1][:300],"...") # Just show first 300 characters

So the values gives us a list of documents that we can analyse.

In [ ]:
# Get a list of documents
documents = list(articles.values())

# View first 400 characters of the 1st document
documents[0][:???]

### Term Count 

**Finding important terms by the frequency of their occurance**

Using `CountVectorizer` create a `vector` for each document where the dimensionality of the vector is the `vocabulary` (all terms in the collection), and the value of each component is the number of times that the `term` occurs in the document.

All of these analyses, approach the document as a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. In this approach, the order of the words don't matter. A popular approach that takes into account order is [Word embedding](https://en.wikipedia.org/wiki/Word_embedding). This session does not explore word embedding.

In [ ]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
count_vectorizer = CountVectorizer(max_df=???,min_df=???,max_features=???,stop_words="english")
count_dt_matrix = count_vectorizer.fit_transform(articles.values())

In [ ]:
# Take a look at the vector for the first document
doc001_vector = count_dt_matrix.toarray()[???]
doc001_vector

In [ ]:
# Get the 1000 terms identified during the vectorization process
feature_names = count_vectorizer.get_feature_names_out()
???

In [ ]:
# Look at how the counts match up to the terms (for the 1st doc)
doc001_term_counts = list(zip(feature_names,???))
doc001_term_counts

In [ ]:
# Take a look at the vocabulary which shows the total counts for whole collection
count_vectorizer.vocabulary_

#### Display matrix in dataframe

Take the term count matrix and display in a dataframe to make visible the structure


In [ ]:
# Create a new dataframe with the matrix - use titles for the index and terms for the columns
count_df = pd.DataFrame(???.toarray(), index=articles.???, columns=???)
count_df

By selecting a row from the dataframe and sorting the values (counts), we can identify the top 10 terms

In [ ]:
# Sample 5 random articles
samples = random.sample(range(0,202),5)

for sample in samples:
    doc = count_df.iloc[???]
    top_terms = dict(count_df.iloc[sample].sort_values(ascending=False).head(???))
    print(f"[{sample}] {doc.name}")
    print("\t- Top terms:",top_terms)

#### Create a top10 terms dataframe

Using the index from the documents, create a dataframe that can hold the top10 terms for each document. We also include columns for our other analysis (tfidf, lda, nmf)

In [ ]:
# Create a dataframe to hold top terms for each analysis type
terms_df = pd.DataFrame(index=count_df.???,columns=['count','tfidf','lda','nmf'])
terms_df

Populate the count column with data created by the count vectorizer.

In [ ]:
#For each doc, get the 10 columns with the largest counts
for idx in terms_df.index:
    counts = dict(???.loc[idx].sort_values(ascending=False).head(???))
    #print(counts)
    terms_df.at[idx,'count'] = list(counts.keys()) # Just the list of terms

terms_df

### Term Frequency / Inverse Document Frequency (TF/IDF)

**Finding terms that are very common in a document, but less common in the whole collection**

The [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) algorithm takes the term frequencies for a document and divides them by the frequencies of the terms in the whole collection.


In [ ]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
tfidf_vectorizer = TfidfVectorizer(
    max_df=???, min_df=???, max_features=???, stop_words="english"
)

In [ ]:
# Get the document vectors
tfidf_dt_matrix = tfidf_vectorizer.fit_transform(???)

# Display the vector for the first document
tfidf_dt_matrix.toarray()[???]

#### Display matrix in dataframe

In [ ]:
tfidf_df = pd.DataFrame(???.toarray(), index=articles.keys(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

#### Update the terms matrix

In [ ]:
for idx in terms_df.index:
    tfidf = dict(tfidf_df.loc[idx].sort_values(ascending=False).head(???))
    #print(counts)
    terms_df.at[idx,'tfidf'] = list(tfidf.keys()) 

terms_df

#### Compare approaches

In [ ]:
# Sample 5 random articles
samples = random.sample(range(0,202),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print()

### Topic modelling with Latent Dirichlet Allocation (LDA)

[LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is an algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It is a generative probabilistic approach to *decomposition* of the document-term matrix into 2 factor matrices: document-topic and topic-term.

![img](https://editor.analyticsvidhya.com/uploads/26864dtm.JPG)

*Source: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/06/part-2-topic-modeling-and-latent-dirichlet-allocation-lda-using-gensim-and-sklearn/)*

The LDA model requires the number of topics to be set in advance. As it is a generative model, it also runs over a number of iterations. These values usually need to be experimented with to obtain quality topics.

In [ ]:
# Set number of topics
num_topics = 20
# Set max number of iteractions
max_iterations = 10

# Create the model
lda_model = LatentDirichletAllocation(n_components=???,max_iter=???,learning_method='online')

# Fit the model to the data, and use the model to transform the data (do the decomposition)
doc_topic_matrix = lda_model.fit_transform(???)

# Obtain the topics
topic_term_matrix = lda_model.components_

#### View the topics

In [ ]:
# Get the topics and their terms
lda_topic_dict = {}
for index, topic in enumerate(???):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    lda_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in lda_topic_dict.items():
    print(k)
    print(v)
    print()

#### List of topics for each document

In [ ]:
doc_topic_matrix[0]

In [ ]:
doc_topic_matrix[200]

#### Update the terms matrix

In [ ]:
for idx,topic in enumerate(???):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['lda'].iloc[idx] = list(top_topic.keys())

terms_df

#### Compare approaches

In [ ]:
# Sample 5 random articles
samples = random.sample(range(0,202),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print()

### Topic modelling with Non-negative Matrix Factorisation (NMF)


[NMF](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a different algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It also factorises the document-term matrix into 2 factor matrices: document-topic and topic-term.

In [ ]:
# Set the number of topics
num_topics = 20

# Create the model
nmf_model = NMF(n_components=???,init='random',beta_loss='frobenius')

# Fit the model to the data and use it to transform the data
doc_topic_nmf = nmf_model.fit_transform(???)

topic_term_nmf = nmf_model.components_

In [ ]:
# Get the topics and their terms
nmf_topic_dict = {}
for index, topic in enumerate(???):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    nmf_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in nmf_topic_dict.items():
    print(k)
    print(v)
    print()

#### Update the terms matrix

In [ ]:
for idx,topic in enumerate(???):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['nmf'].iloc[idx] = list(top_topic.keys())

terms_df

### Compare approaches

In [ ]:
# Sample 5 random articles
samples = random.sample(range(0,202),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print("\t- NMF:\t\t",doc['nmf'])
    print()